In [1]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn import metrics

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
path = './'
train = pd.read_csv(path+'input/train_set.csv')
test = pd.read_csv(path+'input/test_set.csv')
train.shape, test.shape

((25317, 18), (10852, 17))

In [4]:
import matplotlib.pyplot as plt

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

In [6]:
test['y'] = -1
data = train.append(test.reset_index(drop=True))

cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID', 'y']]
for i in cat_col:
#     data['count_' + i] = data.groupby([i])[i].transform('count')
    data[i] = LabelEncoder().fit_transform(data[i].astype(str))
    

In [7]:
feats = [i for i in data.columns if i not in ['ID', 'y']]
# feats

In [8]:
data_train_x = data[data['y'] != -1][feats]
data_train_y = data[data['y'] != -1]['y']
test_x = data[data['y'] == -1][feats]

train_x, valid_x, train_y, valid_y = train_test_split(data_train_x, data_train_y, test_size=0.3)
train_x.shape, train_y.shape, valid_x.shape, valid_y.shape, test_x.shape, data_train_x.shape, data_train_y.shape

((17721, 16),
 (17721,),
 (7596, 16),
 (7596,),
 (10852, 16),
 (25317, 16),
 (25317,))

In [9]:
def measures(test_y, pred):    
    res = [[]]
    res[0].append(metrics.accuracy_score(test_y, pred))
    res[0].append(metrics.precision_score(test_y, pred))
    res[0].append(metrics.recall_score(test_y, pred))
    res[0].append(metrics.f1_score(test_y, pred))
    res[0].append(metrics.roc_auc_score(test_y, pred))
    
    print("accuracy      : %0.3f" % res[0][0])
    print("precision score: %0.3f" % res[0][1])
    print("recall score  : %0.3f" % res[0][2])
    print("f1 score      : %0.3f" % res[0][3])
    print("roc-auc score      : %0.3f" % res[0][4])
    data = pd.DataFrame(res, columns=['accuracy', 'precision', 'recall', 'f1', 'auc'])
    return data

In [25]:
mea_res=pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1', 'auc'])
mea_res

Empty DataFrame
Columns: [accuracy, precision, recall, f1, auc]
Index: []

In [52]:
n_splits = 10
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
train_x = data[data['y']!=-1][feats]
train_y = data[data['y']!=-1]['y']

test_x = data[data['y'] == -1][feats]
res=data[data['y']==-1][['ID']]
res['pred'] = 0

model = lgb.LGBMClassifier(learning_rate=0.4, random_state=42, reg_alpha=0.12,reg_lambda=0.1)

for train_idx, val_idx in kfold.split(train_y):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    valid_x1 = train_x.loc[val_idx]
    valid_y1 = train_y.loc[val_idx]
    
    model.fit(train_x1, train_y1, eval_set=[(train_x1, train_y1),(valid_x1, valid_y1)],
              eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:,1]
res['pred'] = res['pred']/10


[1]	training's auc: 0.888016	training's binary_logloss: 0.26175	valid_1's auc: 0.877562	valid_1's binary_logloss: 0.268865
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.918153	training's binary_logloss: 0.237405	valid_1's auc: 0.909277	valid_1's binary_logloss: 0.246386
[3]	training's auc: 0.925455	training's binary_logloss: 0.22337	valid_1's auc: 0.915425	valid_1's binary_logloss: 0.234814
[4]	training's auc: 0.928822	training's binary_logloss: 0.214343	valid_1's auc: 0.913139	valid_1's binary_logloss: 0.230098
[5]	training's auc: 0.931805	training's binary_logloss: 0.207402	valid_1's auc: 0.915615	valid_1's binary_logloss: 0.224568
[6]	training's auc: 0.937533	training's binary_logloss: 0.198376	valid_1's auc: 0.919955	valid_1's binary_logloss: 0.218009
[7]	training's auc: 0.939153	training's binary_logloss: 0.19412	valid_1's auc: 0.920037	valid_1's binary_logloss: 0.21619
[8]	training's auc: 0.942573	training's binary_logloss: 0.188697	valid_1's

[67]	training's auc: 0.989698	training's binary_logloss: 0.0996852	valid_1's auc: 0.927461	valid_1's binary_logloss: 0.214768
[68]	training's auc: 0.989944	training's binary_logloss: 0.0989872	valid_1's auc: 0.927136	valid_1's binary_logloss: 0.215171
[69]	training's auc: 0.990371	training's binary_logloss: 0.0978517	valid_1's auc: 0.926856	valid_1's binary_logloss: 0.215718
[70]	training's auc: 0.990762	training's binary_logloss: 0.0967941	valid_1's auc: 0.926878	valid_1's binary_logloss: 0.215923
[71]	training's auc: 0.991008	training's binary_logloss: 0.0960434	valid_1's auc: 0.927139	valid_1's binary_logloss: 0.215628
[72]	training's auc: 0.99146	training's binary_logloss: 0.0950649	valid_1's auc: 0.927929	valid_1's binary_logloss: 0.215251
[73]	training's auc: 0.991566	training's binary_logloss: 0.0946245	valid_1's auc: 0.927832	valid_1's binary_logloss: 0.215478
[74]	training's auc: 0.991662	training's binary_logloss: 0.0941124	valid_1's auc: 0.927806	valid_1's binary_logloss: 0.

[33]	training's auc: 0.972601	training's binary_logloss: 0.139585	valid_1's auc: 0.934369	valid_1's binary_logloss: 0.196711
[34]	training's auc: 0.97311	training's binary_logloss: 0.138643	valid_1's auc: 0.934099	valid_1's binary_logloss: 0.196831
[35]	training's auc: 0.973563	training's binary_logloss: 0.137841	valid_1's auc: 0.933314	valid_1's binary_logloss: 0.197418
[36]	training's auc: 0.974121	training's binary_logloss: 0.136583	valid_1's auc: 0.933137	valid_1's binary_logloss: 0.19749
[37]	training's auc: 0.974637	training's binary_logloss: 0.135606	valid_1's auc: 0.93253	valid_1's binary_logloss: 0.198364
[38]	training's auc: 0.975354	training's binary_logloss: 0.134322	valid_1's auc: 0.932244	valid_1's binary_logloss: 0.199346
[39]	training's auc: 0.976448	training's binary_logloss: 0.132533	valid_1's auc: 0.9323	valid_1's binary_logloss: 0.199417
[40]	training's auc: 0.977014	training's binary_logloss: 0.131381	valid_1's auc: 0.932604	valid_1's binary_logloss: 0.198905
[41]	

[1]	training's auc: 0.883389	training's binary_logloss: 0.26437	valid_1's auc: 0.878279	valid_1's binary_logloss: 0.267362
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.914269	training's binary_logloss: 0.241114	valid_1's auc: 0.900292	valid_1's binary_logloss: 0.246765
[3]	training's auc: 0.922518	training's binary_logloss: 0.225246	valid_1's auc: 0.908215	valid_1's binary_logloss: 0.234249
[4]	training's auc: 0.928085	training's binary_logloss: 0.215571	valid_1's auc: 0.911286	valid_1's binary_logloss: 0.227746
[5]	training's auc: 0.931351	training's binary_logloss: 0.208084	valid_1's auc: 0.914411	valid_1's binary_logloss: 0.222339
[6]	training's auc: 0.935065	training's binary_logloss: 0.201691	valid_1's auc: 0.914808	valid_1's binary_logloss: 0.220094
[7]	training's auc: 0.937693	training's binary_logloss: 0.196899	valid_1's auc: 0.917055	valid_1's binary_logloss: 0.217001
[8]	training's auc: 0.939737	training's binary_logloss: 0.193294	valid_

[90]	training's auc: 0.994607	training's binary_logloss: 0.0833487	valid_1's auc: 0.916123	valid_1's binary_logloss: 0.229642
[91]	training's auc: 0.994811	training's binary_logloss: 0.0826631	valid_1's auc: 0.91613	valid_1's binary_logloss: 0.229818
[92]	training's auc: 0.994952	training's binary_logloss: 0.0819789	valid_1's auc: 0.916319	valid_1's binary_logloss: 0.229735
[93]	training's auc: 0.995097	training's binary_logloss: 0.0811918	valid_1's auc: 0.916169	valid_1's binary_logloss: 0.230263
[94]	training's auc: 0.995258	training's binary_logloss: 0.0804451	valid_1's auc: 0.915484	valid_1's binary_logloss: 0.231872
[95]	training's auc: 0.995385	training's binary_logloss: 0.079825	valid_1's auc: 0.914972	valid_1's binary_logloss: 0.232858
[96]	training's auc: 0.995552	training's binary_logloss: 0.0790264	valid_1's auc: 0.915532	valid_1's binary_logloss: 0.232894
[97]	training's auc: 0.995686	training's binary_logloss: 0.0783601	valid_1's auc: 0.915099	valid_1's binary_logloss: 0.2

...

[69]	training's auc: 0.989161	training's binary_logloss: 0.102209	valid_1's auc: 0.928586	valid_1's binary_logloss: 0.205464
[70]	training's auc: 0.989548	training's binary_logloss: 0.101271	valid_1's auc: 0.928453	valid_1's binary_logloss: 0.206229
[71]	training's auc: 0.989924	training's binary_logloss: 0.100108	valid_1's auc: 0.928411	valid_1's binary_logloss: 0.206629
[72]	training's auc: 0.990117	training's binary_logloss: 0.0993605	valid_1's auc: 0.928191	valid_1's binary_logloss: 0.207348
[73]	training's auc: 0.990531	training's binary_logloss: 0.0982605	valid_1's auc: 0.928346	valid_1's binary_logloss: 0.207828
[74]	training's auc: 0.990813	training's binary_logloss: 0.0974658	valid_1's auc: 0.92832	valid_1's binary_logloss: 0.207967
[75]	training's auc: 0.991179	training's binary_logloss: 0.096386	valid_1's auc: 0.928282	valid_1's binary_logloss: 0.208332
[76]	training's auc: 0.991585	training's binary_logloss: 0.0954423	valid_1's auc: 0.928163	valid_1's binary_logloss: 0.2087

[55]	training's auc: 0.985031	training's binary_logloss: 0.112612	valid_1's auc: 0.928051	valid_1's binary_logloss: 0.213503
[56]	training's auc: 0.985469	training's binary_logloss: 0.111546	valid_1's auc: 0.928073	valid_1's binary_logloss: 0.213868
[57]	training's auc: 0.985859	training's binary_logloss: 0.110512	valid_1's auc: 0.92751	valid_1's binary_logloss: 0.214701
[58]	training's auc: 0.986252	training's binary_logloss: 0.109457	valid_1's auc: 0.927852	valid_1's binary_logloss: 0.214607
[59]	training's auc: 0.986518	training's binary_logloss: 0.108674	valid_1's auc: 0.927463	valid_1's binary_logloss: 0.215052
[60]	training's auc: 0.986869	training's binary_logloss: 0.107749	valid_1's auc: 0.927571	valid_1's binary_logloss: 0.215196
[61]	training's auc: 0.98713	training's binary_logloss: 0.106964	valid_1's auc: 0.927635	valid_1's binary_logloss: 0.215052
[62]	training's auc: 0.987383	training's binary_logloss: 0.106106	valid_1's auc: 0.927408	valid_1's binary_logloss: 0.215746
[6

[32]	training's auc: 0.971951	training's binary_logloss: 0.142007	valid_1's auc: 0.925703	valid_1's binary_logloss: 0.201814
[33]	training's auc: 0.972926	training's binary_logloss: 0.140209	valid_1's auc: 0.925553	valid_1's binary_logloss: 0.202301
[34]	training's auc: 0.973886	training's binary_logloss: 0.138556	valid_1's auc: 0.925347	valid_1's binary_logloss: 0.202636
[35]	training's auc: 0.974854	training's binary_logloss: 0.13672	valid_1's auc: 0.925241	valid_1's binary_logloss: 0.203033
[36]	training's auc: 0.97548	training's binary_logloss: 0.135067	valid_1's auc: 0.925766	valid_1's binary_logloss: 0.202722
[37]	training's auc: 0.976294	training's binary_logloss: 0.133465	valid_1's auc: 0.92586	valid_1's binary_logloss: 0.203365
[38]	training's auc: 0.977115	training's binary_logloss: 0.132063	valid_1's auc: 0.926094	valid_1's binary_logloss: 0.203114
[39]	training's auc: 0.977665	training's binary_logloss: 0.130669	valid_1's auc: 0.926234	valid_1's binary_logloss: 0.203163
[40

[1]	training's auc: 0.879827	training's binary_logloss: 0.262543	valid_1's auc: 0.858113	valid_1's binary_logloss: 0.28356
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.913783	training's binary_logloss: 0.237826	valid_1's auc: 0.891222	valid_1's binary_logloss: 0.264097
[3]	training's auc: 0.921096	training's binary_logloss: 0.223848	valid_1's auc: 0.897157	valid_1's binary_logloss: 0.253261
[4]	training's auc: 0.927608	training's binary_logloss: 0.214132	valid_1's auc: 0.902236	valid_1's binary_logloss: 0.247884
[5]	training's auc: 0.931083	training's binary_logloss: 0.206326	valid_1's auc: 0.90549	valid_1's binary_logloss: 0.242785
[6]	training's auc: 0.93657	training's binary_logloss: 0.198795	valid_1's auc: 0.913581	valid_1's binary_logloss: 0.233649
[7]	training's auc: 0.938908	training's binary_logloss: 0.194538	valid_1's auc: 0.917114	valid_1's binary_logloss: 0.230721
[8]	training's auc: 0.941488	training's binary_logloss: 0.190384	valid_1'

[32]	training's auc: 0.973219	training's binary_logloss: 0.137532	valid_1's auc: 0.917923	valid_1's binary_logloss: 0.22875
[33]	training's auc: 0.973904	training's binary_logloss: 0.136241	valid_1's auc: 0.918032	valid_1's binary_logloss: 0.228674
[34]	training's auc: 0.97438	training's binary_logloss: 0.135254	valid_1's auc: 0.917816	valid_1's binary_logloss: 0.229411
[35]	training's auc: 0.975207	training's binary_logloss: 0.133606	valid_1's auc: 0.917215	valid_1's binary_logloss: 0.230202
[36]	training's auc: 0.975749	training's binary_logloss: 0.132478	valid_1's auc: 0.91754	valid_1's binary_logloss: 0.230075
[37]	training's auc: 0.976331	training's binary_logloss: 0.131222	valid_1's auc: 0.917345	valid_1's binary_logloss: 0.230441
[38]	training's auc: 0.977076	training's binary_logloss: 0.129899	valid_1's auc: 0.917017	valid_1's binary_logloss: 0.231354
[39]	training's auc: 0.977719	training's binary_logloss: 0.128661	valid_1's auc: 0.916546	valid_1's binary_logloss: 0.232431
[40

In [53]:
valid_pre = model.predict(train_x)
mea_res = mea_res.append(measures(train_y, valid_pre)).reset_index(drop=True)
mea_res

accuracy      : 0.973
precision score: 0.922
recall score  : 0.842
f1 score      : 0.880
roc-auc score      : 0.916


accuracy  precision    recall        f1       auc
0   0.969783   0.918764  0.813576  0.862977  0.902024
1   0.969783   0.918764  0.813576  0.862977  0.902024
2   0.974168   0.937761  0.834515  0.883131  0.913590
3   0.973496   0.928839  0.837555  0.880838  0.914528
4   0.970652   0.918807  0.821682  0.867534  0.906032
5   0.973575   0.931801  0.835191  0.880855  0.913547
6   0.973022   0.922791  0.839581  0.879222  0.915139
7   0.973259   0.922337  0.842283  0.880494  0.916445
8   0.970771   0.921442  0.819993  0.867763  0.905367
9   0.972904   0.928437  0.832489  0.877849  0.911995
10  0.973694   0.929614  0.838568  0.881747  0.915079
11  0.972272   0.917560  0.838230  0.876103  0.914128
12  0.972864   0.921736  0.839243  0.878558  0.914903
13  0.973259   0.922337  0.842283  0.880494  0.916445

#### 最好记录

In [22]:
n_splits = 10
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
train_x = data[data['y']!=-1][feats]
train_y = data[data['y']!=-1]['y']

test_x = data[data['y'] == -1][feats]
res=data[data['y']==-1][['ID']]
res['pred'] = 0

model = lgb.LGBMClassifier(learning_rate=0.4, random_state=42)

for train_idx, val_idx in kfold.split(train_y):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    valid_x1 = train_x.loc[val_idx]
    valid_y1 = train_y.loc[val_idx]
    
    model.fit(train_x1, train_y1, eval_set=[(train_x1, train_y1),(valid_x1, valid_y1)],
              eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:,1]
res['pred'] = res['pred']/10


[1]	training's auc: 0.887993	training's binary_logloss: 0.261687	valid_1's auc: 0.87847	valid_1's binary_logloss: 0.267401
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.917685	training's binary_logloss: 0.237595	valid_1's auc: 0.909814	valid_1's binary_logloss: 0.245639
[3]	training's auc: 0.925307	training's binary_logloss: 0.223486	valid_1's auc: 0.916242	valid_1's binary_logloss: 0.233383
[4]	training's auc: 0.928644	training's binary_logloss: 0.214254	valid_1's auc: 0.916872	valid_1's binary_logloss: 0.227456
[5]	training's auc: 0.931624	training's binary_logloss: 0.206518	valid_1's auc: 0.91587	valid_1's binary_logloss: 0.223947
[6]	training's auc: 0.936156	training's binary_logloss: 0.198971	valid_1's auc: 0.920449	valid_1's binary_logloss: 0.218263
[7]	training's auc: 0.939417	training's binary_logloss: 0.193493	valid_1's auc: 0.922443	valid_1's binary_logloss: 0.214601
[8]	training's auc: 0.941663	training's binary_logloss: 0.189303	valid_1

[68]	training's auc: 0.99007	training's binary_logloss: 0.0993191	valid_1's auc: 0.924496	valid_1's binary_logloss: 0.219991
[69]	training's auc: 0.990404	training's binary_logloss: 0.0984377	valid_1's auc: 0.924572	valid_1's binary_logloss: 0.220089
[70]	training's auc: 0.990891	training's binary_logloss: 0.0974324	valid_1's auc: 0.924331	valid_1's binary_logloss: 0.220269
[71]	training's auc: 0.991198	training's binary_logloss: 0.0963812	valid_1's auc: 0.924132	valid_1's binary_logloss: 0.220825
[72]	training's auc: 0.991392	training's binary_logloss: 0.0956109	valid_1's auc: 0.924174	valid_1's binary_logloss: 0.221097
[73]	training's auc: 0.991626	training's binary_logloss: 0.0948816	valid_1's auc: 0.923928	valid_1's binary_logloss: 0.221343
[74]	training's auc: 0.991805	training's binary_logloss: 0.0942545	valid_1's auc: 0.923948	valid_1's binary_logloss: 0.221489
[75]	training's auc: 0.991953	training's binary_logloss: 0.0936253	valid_1's auc: 0.924188	valid_1's binary_logloss: 0.

[100]	training's auc: 0.996475	training's binary_logloss: 0.0758172	valid_1's auc: 0.924708	valid_1's binary_logloss: 0.22084
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.996475	training's binary_logloss: 0.0758172	valid_1's auc: 0.924708	valid_1's binary_logloss: 0.22084
[1]	training's auc: 0.88073	training's binary_logloss: 0.266207	valid_1's auc: 0.878294	valid_1's binary_logloss: 0.266915
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.914808	training's binary_logloss: 0.240269	valid_1's auc: 0.902737	valid_1's binary_logloss: 0.245734
[3]	training's auc: 0.921113	training's binary_logloss: 0.225409	valid_1's auc: 0.908836	valid_1's binary_logloss: 0.233749
[4]	training's auc: 0.925478	training's binary_logloss: 0.216757	valid_1's auc: 0.908212	valid_1's binary_logloss: 0.227998
[5]	training's auc: 0.930537	training's binary_logloss: 0.208741	valid_1's auc: 0.910565	valid_1's binary_logloss: 0.223761
[6]	training's auc:

[74]	training's auc: 0.991563	training's binary_logloss: 0.0954755	valid_1's auc: 0.923404	valid_1's binary_logloss: 0.226586
[75]	training's auc: 0.991877	training's binary_logloss: 0.0945986	valid_1's auc: 0.923827	valid_1's binary_logloss: 0.226313
[76]	training's auc: 0.992076	training's binary_logloss: 0.0938432	valid_1's auc: 0.92372	valid_1's binary_logloss: 0.226394
[77]	training's auc: 0.992274	training's binary_logloss: 0.0930509	valid_1's auc: 0.923586	valid_1's binary_logloss: 0.22666
[78]	training's auc: 0.992375	training's binary_logloss: 0.0925223	valid_1's auc: 0.923944	valid_1's binary_logloss: 0.226395
[79]	training's auc: 0.992591	training's binary_logloss: 0.0917199	valid_1's auc: 0.923889	valid_1's binary_logloss: 0.226485
[80]	training's auc: 0.992876	training's binary_logloss: 0.0907692	valid_1's auc: 0.924029	valid_1's binary_logloss: 0.226219
[81]	training's auc: 0.993028	training's binary_logloss: 0.0901058	valid_1's auc: 0.92337	valid_1's binary_logloss: 0.22

[71]	training's auc: 0.991238	training's binary_logloss: 0.0953745	valid_1's auc: 0.913741	valid_1's binary_logloss: 0.234718
[72]	training's auc: 0.99152	training's binary_logloss: 0.0944215	valid_1's auc: 0.913858	valid_1's binary_logloss: 0.234987
[73]	training's auc: 0.991822	training's binary_logloss: 0.0935563	valid_1's auc: 0.9139	valid_1's binary_logloss: 0.235226
[74]	training's auc: 0.992089	training's binary_logloss: 0.0927702	valid_1's auc: 0.914012	valid_1's binary_logloss: 0.235762
[75]	training's auc: 0.992269	training's binary_logloss: 0.0921404	valid_1's auc: 0.914315	valid_1's binary_logloss: 0.235829
[76]	training's auc: 0.992362	training's binary_logloss: 0.0915056	valid_1's auc: 0.914374	valid_1's binary_logloss: 0.235952
[77]	training's auc: 0.992637	training's binary_logloss: 0.0906581	valid_1's auc: 0.913925	valid_1's binary_logloss: 0.236445
[78]	training's auc: 0.992917	training's binary_logloss: 0.0897948	valid_1's auc: 0.913283	valid_1's binary_logloss: 0.23

...

[1]	training's auc: 0.888677	training's binary_logloss: 0.262597	valid_1's auc: 0.854896	valid_1's binary_logloss: 0.261423
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.917286	training's binary_logloss: 0.239326	valid_1's auc: 0.88687	valid_1's binary_logloss: 0.246155
[3]	training's auc: 0.922617	training's binary_logloss: 0.225522	valid_1's auc: 0.893736	valid_1's binary_logloss: 0.234342
[4]	training's auc: 0.928794	training's binary_logloss: 0.215131	valid_1's auc: 0.901368	valid_1's binary_logloss: 0.227067
[5]	training's auc: 0.932681	training's binary_logloss: 0.207431	valid_1's auc: 0.907786	valid_1's binary_logloss: 0.221038
[6]	training's auc: 0.938686	training's binary_logloss: 0.198467	valid_1's auc: 0.915501	valid_1's binary_logloss: 0.214094
[7]	training's auc: 0.941667	training's binary_logloss: 0.193636	valid_1's auc: 0.918041	valid_1's binary_logloss: 0.211273
[8]	training's auc: 0.943796	training's binary_logloss: 0.189696	valid_

[69]	training's auc: 0.990469	training's binary_logloss: 0.0989401	valid_1's auc: 0.920377	valid_1's binary_logloss: 0.216077
[70]	training's auc: 0.990799	training's binary_logloss: 0.0980898	valid_1's auc: 0.920598	valid_1's binary_logloss: 0.216205
[71]	training's auc: 0.991017	training's binary_logloss: 0.0973271	valid_1's auc: 0.920923	valid_1's binary_logloss: 0.215769
[72]	training's auc: 0.991221	training's binary_logloss: 0.0964643	valid_1's auc: 0.921225	valid_1's binary_logloss: 0.21585
[73]	training's auc: 0.991305	training's binary_logloss: 0.0960812	valid_1's auc: 0.921426	valid_1's binary_logloss: 0.215578
[74]	training's auc: 0.991503	training's binary_logloss: 0.0952922	valid_1's auc: 0.921366	valid_1's binary_logloss: 0.215907
[75]	training's auc: 0.991921	training's binary_logloss: 0.0941835	valid_1's auc: 0.92179	valid_1's binary_logloss: 0.215985
[76]	training's auc: 0.992219	training's binary_logloss: 0.0932553	valid_1's auc: 0.921361	valid_1's binary_logloss: 0.2

[33]	training's auc: 0.974637	training's binary_logloss: 0.135636	valid_1's auc: 0.920496	valid_1's binary_logloss: 0.22819
[34]	training's auc: 0.975211	training's binary_logloss: 0.13455	valid_1's auc: 0.919945	valid_1's binary_logloss: 0.229264
[35]	training's auc: 0.975738	training's binary_logloss: 0.133461	valid_1's auc: 0.91944	valid_1's binary_logloss: 0.230297
[36]	training's auc: 0.976305	training's binary_logloss: 0.132228	valid_1's auc: 0.918886	valid_1's binary_logloss: 0.23127
[37]	training's auc: 0.977098	training's binary_logloss: 0.130635	valid_1's auc: 0.918926	valid_1's binary_logloss: 0.231526
[38]	training's auc: 0.977745	training's binary_logloss: 0.129235	valid_1's auc: 0.919174	valid_1's binary_logloss: 0.230716
[39]	training's auc: 0.978432	training's binary_logloss: 0.127874	valid_1's auc: 0.918726	valid_1's binary_logloss: 0.231751
[40]	training's auc: 0.979289	training's binary_logloss: 0.126364	valid_1's auc: 0.919327	valid_1's binary_logloss: 0.230671
[41]

[1]	training's auc: 0.885499	training's binary_logloss: 0.259923	valid_1's auc: 0.86567	valid_1's binary_logloss: 0.283744
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.919128	training's binary_logloss: 0.236406	valid_1's auc: 0.900334	valid_1's binary_logloss: 0.262859
[3]	training's auc: 0.92572	training's binary_logloss: 0.222225	valid_1's auc: 0.904784	valid_1's binary_logloss: 0.251038
[4]	training's auc: 0.929653	training's binary_logloss: 0.212522	valid_1's auc: 0.905393	valid_1's binary_logloss: 0.24557
[5]	training's auc: 0.932433	training's binary_logloss: 0.206046	valid_1's auc: 0.90957	valid_1's binary_logloss: 0.240255
[6]	training's auc: 0.936054	training's binary_logloss: 0.199164	valid_1's auc: 0.911542	valid_1's binary_logloss: 0.235598
[7]	training's auc: 0.93989	training's binary_logloss: 0.193816	valid_1's auc: 0.911631	valid_1's binary_logloss: 0.233198
[8]	training's auc: 0.943316	training's binary_logloss: 0.18818	valid_1's a

[96]	training's auc: 0.994212	training's binary_logloss: 0.101537	valid_1's auc: 0.911779	valid_1's binary_logloss: 0.268091
[97]	training's auc: 0.995003	training's binary_logloss: 0.0811444	valid_1's auc: 0.911999	valid_1's binary_logloss: 0.267792
[98]	training's auc: 0.995136	training's binary_logloss: 0.0804975	valid_1's auc: 0.912135	valid_1's binary_logloss: 0.267657
[99]	training's auc: 0.995317	training's binary_logloss: 0.0797905	valid_1's auc: 0.911965	valid_1's binary_logloss: 0.268143
[100]	training's auc: 0.995493	training's binary_logloss: 0.0788576	valid_1's auc: 0.912293	valid_1's binary_logloss: 0.268635
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.995493	training's binary_logloss: 0.0788576	valid_1's auc: 0.912293	valid_1's binary_logloss: 0.268635


In [23]:
valid_pre = model.predict(train_x)
mea_res = measures(train_y, valid_pre)

accuracy      : 0.972
precision score: 0.924
recall score  : 0.827
f1 score      : 0.873
roc-auc score      : 0.909


accuracy  precision    recall        f1       auc
0  0.971837   0.924471  0.826748  0.872883  0.908901

In [13]:
res.to_csv('/python/datasource/submisission.csv', index=False)

In [15]:
lgb.LGBMClassifier?

### baseline